In [1]:
import networkx as nx
import axelrod as axl
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
import numpy as np
import random
%matplotlib inline
from networkx.drawing.nx_agraph import graphviz_layout
#import statsmodels.formula.api as sm

In [3]:
players = [axl.Gambler(), axl.EvolvedLookerUp(), axl.DoubleCrosser(), axl.FoolMeOnce(),
           axl.Gradual(), axl.BackStabber(), axl.ForgetfulFoolMeOnce(), axl.MetaHunter(),
           axl.OmegaTFT(), axl.MetaMajorityLongMemory(), axl.MetaMajorityFiniteMemory(),
           axl.Grudger(), axl.MetaMajorityMemoryOne(), axl.MetaWinner(), axl.NiceAverageCopier(),
           axl.Shubik(), axl.ZDGTFT2(), axl.SoftGrudger(), axl.SoftJoss(), axl.LimitedRetaliate2()]

In [4]:
def Neighborhood(G):
    Neighborhood = []
    for i in range(len(players)) : Neighborhood.append(G.neighbors(i))
    return Neighborhood

In [5]:
def Neighborhood_size (Neighborhood) :
    Neighborhood_size = []
    for i in range(len(Neighborhood)) : Neighborhood_size.append(len(Neighborhood[i]))
    return Neighborhood_size

In [6]:
def NeighborsScores(G, results):
    return [[results.scores[j] for j in G.neighbors(i)]
            for i, pl in enumerate(players)]

In [7]:
def AvNeighborsScore(G, results) :
    av_score = []
    temp = [[j for m in G.neighbors(k) for j in results.scores[m]]
              for k in G.nodes()]
    for i in G.nodes() :
        av_score.append(np.mean(temp[i]))
    return av_score

In [30]:
def touranment(G, p, seed):
    """Run a random tournament with a binomial 
       structure.
    """
    random.seed(seed)
    np.random.seed(seed)
    edges = G.edges()
    tournament = axl.SpatialTournament(players, edges=G.edges(), turns = 200, repetitions=5)
    return tournament.play(processes=0,
                           filename="/home/nikoleta/src/Dissertation-Notebooks/Data/Watts Strogatz/Watts_Strogatz-8-{}-{}.csv".format(p, seed))

In [31]:
def tournament_results(G, seed,p):
    results = touranment(G, p, seed)
    neighborhood = Neighborhood(G)
    nsize = Neighborhood_size(neighborhood)
    Degree = list(G.degree(G.nodes()).values())
    nscores = NeighborsScores(G, results)
    av_nscores = AvNeighborsScore(G, results)
    
    data = pd.DataFrame({'seed' : seed, 'Parameter': p ,'PlayersIndex' : G.nodes(),
                         'PlayersName' : results.players, 'Degree' : Degree ,
                         'Neighborhood' : neighborhood, 'NeighborhoodSize' : nsize,
                         'Ranking' : results.ranking, 'Scores' : results.scores,
                         'NormalisedScores' : results.normalised_scores,
                         'AverageScore' : [np.median(scores) for scores in results.normalised_scores],
                         'NeighborsScore' : nscores, 'AverageNeighborsScore' : av_nscores,
                         #'Payoffs' : tournament.game.RPST(),
                         'Connectivity' : nx.node_connectivity(G), 
                         'Clustering' : nx.average_clustering(G)

                        })
    return data

In [32]:
results = pd.DataFrame()
for seed in range(0, 100):
    
    np.random.seed(seed)
    p = seed/100
    G = nx.watts_strogatz_graph (len(players), 8, p)
    results = results.append([tournament_results(G, seed,p)])

results.to_csv("/home/nikoleta/src/Dissertation-Notebooks/Data/Watts Strogatz/Watts_Strogatz_Eight_Tournament.csv", index=False) 


In [15]:
results

,AverageNeighborsScore,AverageScore,Clustering,Connectivity,Degree,Neighborhood,NeighborhoodSize,NeighborsScore,NormalisedScores,Parameter,PlayersIndex,PlayersName,Ranking,Scores,seed
0,4639.000000,2.435000,0.102619,4,4,"[76, 41, 36, 53]",4,"[[3030, 3030, 3030, 3030, 3030], [4294, 4294, ...","[2.435, 2.435, 2.435, 2.435, 2.435]",0.10,0,Adaptive,20,"[1948, 1948, 1948, 1948, 1948]",10
1,4938.700000,1.732500,0.102619,4,10,"[64, 81, 36, 37, 84, 24, 9, 44, 29, 53]",10,"[[5300, 5300, 5300, 5300, 5300], [6455, 6455, ...","[1.7325, 1.7325, 1.7325, 1.7325, 1.7325]",0.10,1,Aggravater,26,"[3465, 3465, 3465, 3465, 3465]",10
2,4976.000000,1.235000,0.102619,4,10,"[49, 52, 90, 73, 23, 72, 41, 10, 61, 79]",10,"[[5454, 5454, 5454, 5454, 5454], [3585, 3585, ...","[1.235, 1.235, 1.235, 1.235, 1.235]",0.10,2,Alternator,90,"[2470, 2470, 2470, 2470, 2470]",10
3,5272.700000,1.902000,0.102619,4,10,"[61, 19, 70, 22, 10, 43, 44, 26, 46, 47]",10,"[[4627, 4627, 4627, 4627, 4627], [4645, 4645, ...","[1.902, 1.902, 1.902, 1.902, 1.902]",0.10,3,Alternator Hunter,58,"[3804, 3804, 3804, 3804, 3804]",10
4,5134.307692,2.080385,0.102619,4,13,"[83, 68, 69, 11, 80, 19, 20, 53, 22, 25, 24, 5...",13,"[[3801, 3801, 3801, 3801, 3801], [3651, 3651, ...","[2.08038461538, 2.08038461538, 2.08038461538, ...",0.10,4,AntiCycler,30,"[5409, 5409, 5409, 5409, 5409]",10
5,4296.538462,1.900385,0.102619,4,13,"[32, 64, 66, 68, 71, 40, 31, 78, 80, 53, 56, 5...",13,"[[3528, 3528, 3528, 3528, 3528], [5300, 5300, ...","[1.90038461538, 1.90038461538, 1.90038461538, ...",0.10,5,Anti Tit For Tat,54,"[4941, 4941, 4941, 4941, 4941]",10
6,4941.777778,2.978333,0.102619,4,9,"[65, 35, 68, 39, 24, 84, 26, 27, 28]",9,"[[2995, 2995, 2995, 2995, 2995], [6257, 6257, ...","[2.97833333333, 2.97833333333, 2.97833333333, ...",0.10,6,Adapative Pavlov 2006,47,"[5361, 5361, 5361, 5361, 5361]",10
7,4713.375000,2.524375,0.102619,4,8,"[50, 52, 55, 57, 63, 12, 62, 47]",8,"[[4910, 4910, 4910, 4910, 4910], [3585, 3585, ...","[2.524375, 2.524375, 2.524375, 2.524375, 2.524...",0.10,7,Adapative Pavlov 2011,45,"[4039, 4039, 4039, 4039, 4039]",10
8,5005.571429,2.599286,0.102619,4,14,"[80, 11, 38, 70, 81, 72, 75, 14, 16, 40, 86, 5...",14,"[[6373, 6373, 6373, 6373, 6373], [4730, 4730, ...","[2.59928571429, 2.59928571429, 2.59928571429, ...",0.10,8,Appeaser,52,"[7278, 7278, 7278, 7278, 7278]",10
9,4213.000000,2.598000,0.102619,4,5,"[64, 1, 83, 59, 29]",5,"[[5300, 5300, 5300, 5300, 5300], [3465, 3465, ...","[2.598, 2.598, 2.598, 2.598, 2.598]",0.10,9,BackStabber,6,"[2598, 2598, 2598, 2598, 2598]",10
